In [1]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('./KC_case_data .zip', parse_dates=['date'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   event         object        
 2   purchase_sum  float64       
 3   os_name       object        
 4   device_id     int64         
 5   gender        object        
 6   city          object        
 7   utm_source    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 167.7+ MB


In [17]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


## Задание 4
MAU февраля

In [19]:
df.resample(rule='M', on='date').agg({'device_id':'nunique'})

,device_id
date,
2020-01-31,99161
2020-02-29,75032
2020-03-31,74623


## Задание 5
Количество установок в январе

In [20]:
df.event.unique()

array(['app_start', 'choose_item', 'purchase', 'search', 'tap_basket',
       'app_install', 'register'], dtype=object)

In [21]:
df.query('event == "app_install"').resample(rule='M', on='date').agg({'device_id':'count'})

,device_id
date,
2020-01-31,80297
2020-02-29,38078
2020-03-31,36222


## Задание 6
Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [46]:
df_preretantion = df.groupby(by=['device_id','event'], as_index=False) \
    .agg({'date':'min'}) \
    .pivot(index='device_id', columns=['event'], values='date')

In [55]:
df_preretantion['cohort_day'] = df_preretantion.app_install

In [66]:
df_preretantion['purchase_day'] = df_preretantion.purchase - df_preretantion.cohort_day
df_preretantion['purchase_day'] = df_preretantion.purchase_day.dt.days

In [71]:
df_preretantion = df_preretantion.drop(columns=['app_install', 'app_start', 'choose_item', 'register', 'search', 'tap_basket'])

In [75]:
df_preretantion = df_preretantion.drop(df_preretantion[df_preretantion.cohort_day.isna()].index)

In [79]:
df_preretantion['is_converted'] = df_preretantion.purchase_day <= 7

In [99]:
df_cr_7day = df_preretantion.groupby(by='cohort_day') \
    .is_converted.value_counts(normalize=True) \
    .to_frame() \
    .rename(columns={'is_converted':'cr'}) \
    .reset_index() \
    .query('is_converted') \
    .sort_values(by='cr', ascending=False)

In [102]:
df_cr_7day.iloc[0]

cohort_day      2020-01-01 00:00:00
is_converted                   True
cr                         0.393406
cr_rounded                     39.3
Name: 1, dtype: object

## Задание 7
Укажите значение CR из предыдущего вопроса в % (округлите до 1 цифры после запятой). Формат числа - с точкой. Пример: 66.4.


In [100]:
df_cr_7day['cr_rounded'] = np.round(df_cr_7day.cr * 100, 1)

In [103]:
df_cr_7day.iloc[0]

cohort_day      2020-01-01 00:00:00
is_converted                   True
cr                         0.393406
cr_rounded                     39.3
Name: 1, dtype: object

## Задание 8
С какого платного маркетингового канала пришло больше всего новых пользователей? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

NB! В этом и подобном ему заданиях писать надо только название, без кавычек.

In [108]:
df.groupby('utm_source') \
    .agg({'device_id':'nunique'}) \
    .sort_values(by='device_id')

,device_id
utm_source,
referal,15926
facebook_ads,25959
instagram_ads,31048
vk_ads,34086
google_ads,38096
yandex-direct,40712
-,52273


## Задание 9
Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

Подсказка: 

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 

In [118]:
df_register = df.query('event == "register"') \
    .groupby(by='device_id', as_index=False) \
    .agg({'date':'min'}) \
    .rename(columns={'date':'registaration_date'})

df = pd.merge(df, df_register, on='device_id')

In [212]:
df.event.unique()

array(['app_start', 'search', 'choose_item', 'tap_basket', 'purchase',
       'app_install', 'register'], dtype=object)

In [213]:
map = {
    # 'app_install' : '1_app_install',
    # 'app_start' : '2_app_start',
    'search' : '3_search',
    'choose_item' : '4_choose_item',
    'tap_basket' : '5_tap_basket',
    'register' : '6_register',
    'purchase' : '7_purchase'
}

In [214]:
df['funnel_order'] = df.event.map(map)

In [215]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,registaration_date,funnel_order
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07,NaN
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,2020-01-07,3_search
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,2020-01-07,NaN
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,2020-01-07,4_choose_item
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,2020-01-07,3_search


In [216]:
def get_funnel(data):
    funnel = data.groupby(by='funnel_order', as_index=False) \
    .agg({'device_id':'count'}) \
    .rename(columns={'device_id':'_count'})

    funnel['pct'] = 1 - (funnel._count.pct_change() * -1)

    return funnel.fillna(1)

In [217]:
df_funnel_registered = get_funnel(df.query('registaration_date < date'))
df_funnel_registered

,funnel_order,_count,pct
0,3_search,417101,1.000000
1,4_choose_item,314473,0.753949
2,5_tap_basket,217256,0.690857
3,7_purchase,73630,0.338909


In [218]:
df_funnel_not_registered = get_funnel(df.query('registaration_date >= date'))
df_funnel_not_registered

,funnel_order,_count,pct
0,3_search,148170,1.000000
1,4_choose_item,125994,0.850334
2,5_tap_basket,105031,0.833619
3,6_register,78310,0.745589
4,7_purchase,67753,0.865190


## Задание 10
Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

Подсказка: 

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [236]:
df_pivot = df.groupby(by=['utm_source', 'event'], as_index=False) \
    .agg({'device_id':'nunique'}) \
    .rename(columns={'device_id':'counts'}) \
    .pivot(index='utm_source', columns='event', values='counts')

df_pivot

event,app_install,app_start,choose_item,purchase,register,search,tap_basket
utm_source,,,,,,,
-,13330,23653,21991,16598,16527,23332,20614
facebook_ads,5711,12519,11711,9017,8672,12375,11035
google_ads,10859,16896,15510,11339,11405,16661,14358
instagram_ads,8062,14869,13824,10762,10750,14676,13005
referal,3821,8411,7904,6362,6193,8325,7494
vk_ads,9729,16852,15748,12364,12421,16645,14839
yandex-direct,12014,17954,16508,12028,12342,17706,15313


In [237]:
(df_pivot.purchase / df_pivot.app_start) \
    .to_frame() \
    .reset_index() \
    .rename(columns={0:'cr'}) \
    .sort_values(by='cr')

,utm_source,cr
6,yandex-direct,0.669934
2,google_ads,0.671106
0,-,0.701729
1,facebook_ads,0.720265
3,instagram_ads,0.723788
5,vk_ads,0.733681
4,referal,0.756390


## Задание 11
Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

In [259]:
df_purchase = df.query('event == "purchase"') \
    .groupby(by='device_id', as_index=False) \
    .agg({'date':'min'}) \
    .rename(columns={'date':'purchase_date'})

df = pd.merge(df, df_purchase, on='device_id')

In [260]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,funnel_order,registaration_date,purchase_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,NaN,2020-01-07,2020-01-07
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,3_search,2020-01-07,2020-01-07
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,NaN,2020-01-07,2020-01-07
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,4_choose_item,2020-01-07,2020-01-07
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,3_search,2020-01-07,2020-01-07


In [271]:
df['purchase_date'] = df.query('event == "purchase"').groupby(by='device_id').date.transform('min')

df.query('event == "purchase" and date == purchase_date').groupby(by='utm_source') \
    .agg({'purchase_sum':'median'}) \
    .sort_values(by='purchase_sum')

,purchase_sum
utm_source,
facebook_ads,389.0
google_ads,390.5
yandex-direct,392.5
vk_ads,393.0
instagram_ads,393.5
referal,395.5
-,398.5


# Задание 12
Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Какой платный канал привлечения имеет самый высокий ROMI? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

In [272]:
map_cost = {
    'yandex-direct' : 10491707,
    'google_ads' : 10534878,
    'facebook_ads' : 8590498,
    'instagram_ads' : 8561626,
    'vk_ads': 9553531
}

In [274]:
df_romi = df.groupby('utm_source', as_index=False).agg({'purchase_sum':'sum'})

In [275]:
df_romi['cost'] = df_romi.utm_source.map(map_cost)

In [277]:
df_romi['romi'] = (df_romi.purchase_sum - df_romi.cost) / df_romi.cost

In [280]:
df_romi.sort_values('romi', ascending=False)

,utm_source,purchase_sum,cost,romi
5,vk_ads,16389652.5,9553531.0,0.715560
3,instagram_ads,14546969.0,8561626.0,0.699090
1,facebook_ads,12249901.0,8590498.0,0.425983
6,yandex-direct,13915368.0,10491707.0,0.326321
2,google_ads,12868276.0,10534878.0,0.221493
0,-,21449749.5,NaN,NaN
4,referal,8837044.5,NaN,NaN
